In [0]:
# Databricks notebook source
# MAGIC %run /Users/eh163e@att.com/utl/set_service_principal

# COMMAND ----------

import pandas as pd
from functools import reduce
from pyspark.sql import DataFrame
from datetime import datetime

# COMMAND ----------

# MAGIC %md
# MAGIC # Read in DataWeb Whitelist and Blocklist

# COMMAND ----------

date1 = '20220128'
date2 = str(datetime.now()).replace("-","")
mydates = pd.date_range(date1, date2).tolist()

# COMMAND ----------

run_date = str(mydates[0].date())
root = "abfss://robofeed@famlisandbox.dfs.core.windows.net/robo_rpts/"
mobility_block_path = root +f"research.{run_date}.MobilityBLOCK"
whitelist_path = root +f"research.{run_date}.Whitelist"
mobility_flagged_path1 = root +f"research.{run_date}.RobocallMobility"
mobility_flagged_path2 = root +f"research.{run_date}.RobocallMobility-8YY"
mobility_unblock_path = root +f"research.{run_date}.MobilityUNBLOCK"
mobility_block_activity_path = root +f"research.{run_date}.MobilityBlockActivity"
vm_pilot_path = 'abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/eh163e/data/robocalling/VM_ID_Pilot_Numbersd.csv'

cnam_path =root+"cnamcache.2022-02-21"
# cnam = (spark
#   .read
#   .format("CSV")
#   .option("header",True)
#   .option("delimiter","|")
#   .load(cnam_path))

# COMMAND ----------

whitelist = (spark
  .read
  .format("CSV")
  .option("header",True)
  .option("delimiter","|")
  .load(whitelist_path)
  .toPandas())
whitelist_total_lines = whitelist.iloc[0][2]
whitelist.columns = ["Number","cnam","date","_c3"]
whitelist = whitelist[1:]

# COMMAND ----------

#whitelist = whitelist[1:]
whitelist.date = pd.to_datetime(whitelist.date,errors='coerce')
whitelist.date.max()

# COMMAND ----------

whitelist


# COMMAND ----------

mobility_unblock["Date"] = pd.to_datetime(mobility_unblock["Date"])
mobility_unblock["date"] = mobility_unblock["Date"].astype(str)
mobility_unblock["feed"]= "unblock"
mobility_unblock

# COMMAND ----------

mobility_block["date"]= run_date
mobility_block["feed"]= "block"

mobility_block.head()

# COMMAND ----------

all_dfs = []

for date in mydates[0:2]:
  run_date = str(date.date())
  root = "abfss://robofeed@famlisandbox.dfs.core.windows.net/robo_rpts/"
  mobility_block_path = root +f"research.{run_date}.MobilityBLOCK"
  whitelist_path = root +f"research.{run_date}.Whitelist"
  mobility_flagged_path1 = root +f"research.{run_date}.RobocallMobility"
  mobility_flagged_path2 = root +f"research.{run_date}.RobocallMobility-8YY"
  mobility_unblock_path = root +f"research.{run_date}.MobilityUNBLOCK"
  #mobility_block_activity_path = root +f"research.{run_date}.MobilityBlockActivity"
  vm_pilot_path = 'abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/eh163e/data/robocalling/VM_ID_Pilot_Numbersd.csv'

  mobility_block = (spark
    .read
    .format("CSV")
    .option("header",False)
    .option("delimiter","|")
    .load(mobility_block_path)
    .toPandas())
  mobility_block_totals = list(mobility_block.iloc[0])
  mobility_block.columns = list(mobility_block.iloc[1])
  mobility_block = mobility_block[2:]
  mobility_block["run_date"]= run_date # take max
  mobility_block["date"]= run_date # take min
  mobility_block["feed"]= "block"
  all_dfs.append(mobility_block[['Number', 'date','feed',"run_date"]])

#   mobility_block_activity = (spark
#     .read
#     .format("CSV")
#     .option("header",False)
#     .option("delimiter","|")
#     .load(mobility_block_activity_path)
#     .toPandas())
#   mobility_block_activity_totals = list(mobility_block_activity.iloc[0])
#   mobility_block_activity.columns = list(mobility_block_activity.iloc[1])
#   mobility_block_activity = mobility_block_activity[2:]

  whitelist = (spark
    .read
    .format("CSV")
    .option("header",True)
    .option("delimiter","|")
    .load(whitelist_path)
    .toPandas())
  whitelist_total_lines = whitelist.iloc[0][2]
  whitelist.columns = ["Number","cnam","date","_c3"]
  whitelist = whitelist[1:]
  whitelist["run_date"] = run_date
  whitelist["feed"]= "whitelist"
  all_dfs.append(whitelist[['Number', 'date','feed',"run_date"]])

  mobility_unblock = (spark
    .read
    .format("CSV")
    .option("header",True)
    .option("delimiter","|")
    .load(mobility_unblock_path)
    .toPandas())
  mobility_unblock["Date"] = pd.to_datetime(mobility_unblock["Date"])
  mobility_unblock["run_date"] = run_date
  mobility_unblock["date"] = mobility_unblock["Date"].astype(str)
  mobility_unblock["feed"]= "unblock"
  all_dfs.append(mobility_unblock[['Number', 'date','feed',"run_date"]])

  mobility_flagged1 = (spark
    .read
    .format("CSV")
    .option("header",False)
    .option("delimiter","|")
    .load(mobility_flagged_path1)
    .toPandas())
  mobility_flagged1_totals = list(mobility_flagged1.iloc[0])
  mobility_flagged1.columns = list(mobility_flagged1.iloc[1])
  mobility_flagged1 = mobility_flagged1[2:]
  mobility_flagged1.columns =['Number', 'num_of_OPCs', 'LNP_Lookup', 'num_of_Calls','DNC']
  mobility_flagged1["run_date"]= run_date
  mobility_flagged1["date"]= run_date
  mobility_flagged1["feed"]= "flagged_mobility"
  all_dfs.append(mobility_flagged1[['Number', 'date','feed',"run_date"]])

  mobility_flagged2 = (spark
    .read
    .format("CSV")
    .option("header",False)
    .option("delimiter","|")
    .load(mobility_flagged_path2)
    .select("_c0","_c1","_c3","_c4")
    .toPandas())
  mobility_flagged2_totals = list(mobility_flagged2.iloc[0])
  mobility_flagged2.columns = list(mobility_flagged2.iloc[1])
  mobility_flagged2 = mobility_flagged2[2:]
  mobility_flagged2.columns =['Number', 'num_of_OPCs','num_of_Calls','DNC']
  mobility_flagged2["run_date"]= run_date
  mobility_flagged2["date"]= run_date
  mobility_flagged2["feed"]= "flagged_mobility8YY"
  all_dfs.append(mobility_flagged2[['Number', 'date','feed',"run_date"]])

  vm_pilot = (spark
    .read
    .format("CSV")
    .option("header",True)
    .option("delimiter",",")
    .load(vm_pilot_path)
    .select("DEPOSIT_NUMBER")  
    .withColumnRenamed("DEPOSIT_NUMBER","ctn")
    .toPandas())

# COMMAND ----------

full_df = pd.concat(all_dfs)
full_df.date = pd.to_datetime(full_df.date,errors='coerce')
full_df.run_date = pd.to_datetime(full_df.run_date,errors='coerce')

# COMMAND ----------

full_df_agg = full_df.groupby(["Number","feed"]).agg(['min', 'max'])
full_df_agg

# COMMAND ----------

full_df_agg.reset_index()

# COMMAND ----------



from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

historical_robo_labels_path = "abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/eh163e/data/robocalling/historical_robo_features_with_labels3"

files = dbutils.fs.ls("abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/call_graph/source/daily_refresh/")
all_dates = [file[1][:-1] for file in files if len(file[1]) > 7]

# COMMAND ----------

# add check condition on historical trusted edges date to see if already updated
try:
  historical_robocalling = (spark
     .read
     .format('delta')
     .load(historical_robo_labels_path))
  historical_dates = historical_robocalling.select("Date").distinct().collect()
except Exception as e:
    #print(e)
    historical_dates = []

#get dates in historical robocall labels
if len(historical_dates) > 0:
  historical_dates1 = [row['Date'] for row in historical_dates]
  #subset dates to get dates not in historical table
  dates_notin = [dt for dt in all_dates if (dt not in historical_dates1) & (dt[0]=="2")]
  print("Dates data not present")
  print(dates_notin)

# COMMAND ----------

# iterate through the files and append them to an aggregated delta table for all robocall 
if len(dates_notin) > 0:
  for date in dates_notin:
    print("Proccessing Date:",date)
    if int(date) < 20220128:
      # if date is before jan 28 when we don't have dataweb
      try:
        df = (spark
              .read
              .format("csv")
              .option("header",True)
              .load(f"abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/call_graph/source/daily_refresh/{date}/features_extracted/")
            .withColumn("outgoing_call_count", col("outgoing_call_count").cast(IntegerType()))
            .withColumn("incoming_call_count", col("incoming_call_count").cast(IntegerType()))
            .withColumn("incoming_to_outgoing_ratio", col("incoming_to_outgoing_ratio").cast(DoubleType()))
            .withColumn("outgoing_untrusted_call_count", col("outgoing_untrusted_call_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_call_count", col("outgoing_trusted_call_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_call_ratio", col("outgoing_trusted_call_ratio").cast(DoubleType()))
            .withColumn("call_volume", col("call_volume").cast(IntegerType()))
            .withColumn("otc_in_ratio", col("otc_in_ratio").cast(DoubleType()))
            .withColumn("unique_ctn_called", col("unique_ctn_called").cast(IntegerType()))
            .withColumn("mean_degree_one_outgoing_calls", col("mean_degree_one_outgoing_calls").cast(DoubleType()))
            .withColumn("trusted_edge_count", col("trusted_edge_count").cast(IntegerType()))
            .withColumn("outgoing_text_count", col("outgoing_text_count").cast(IntegerType()))
            .withColumn("incoming_text_count", col("incoming_text_count").cast(IntegerType()))
            .withColumn("incoming_to_outgoing_ratio_text", col("incoming_to_outgoing_ratio_text").cast(DoubleType()))
            .withColumn("outgoing_untrusted_text_count", col("outgoing_untrusted_text_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_text_count", col("outgoing_trusted_text_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_text_ratio", col("outgoing_trusted_text_ratio").cast(DoubleType()))
            .withColumn("unique_ctn_texted", col("unique_ctn_texted").cast(IntegerType()))
            .withColumn("mean_degree_one_outgoing_texts", col("mean_degree_one_outgoing_texts").cast(DoubleType()))
            .withColumn("volume_cat", col("volume_cat").cast(IntegerType()))
            .withColumn("repeated_calls", col("repeated_calls").cast(DoubleType()))
            .withColumn("repeated_texts", col("repeated_texts").cast(DoubleType()))
            .withColumn("text_volume", col("text_volume").cast(IntegerType()))
            .withColumn("ott_in_ratio", col("ott_in_ratio").cast(DoubleType()))
            .withColumn("risk_cat", col("risk_cat").cast(IntegerType()))
            .withColumn("Date", date_format(to_date(col("Date")),"yyyyMMdd"))
            ).fillna(0).fillna(0.0).fillna(False)
        df.write.option("header",True).partitionBy("Date").option("mergeSchema", "true").format("delta").mode("append").save(historical_robo_labels_path)
      except Exception as e:
        print(e)
    else:
      run_date = str(datetime.strptime(str(date), '%Y%m%d').date())
      root = "abfss://robofeed@famlisandbox.dfs.core.windows.net/robo_rpts/"
      mobility_block_path = root +f"research.{run_date}.MobilityBLOCK"
      whitelist_path = root +f"research.{run_date}.Whitelist"
      mobility_flagged_path1 = root +f"research.{run_date}.RobocallMobility"
      mobility_flagged_path2 = root +f"research.{run_date}.RobocallMobility-8YY"
      mobility_unblock_path = root +f"research.{run_date}.MobilityUNBLOCK"
      mobility_block_activity_path = root +f"research.{run_date}.MobilityBlockActivity"
      vm_pilot_path = 'abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/eh163e/data/robocalling/VM_ID_Pilot_Numbersd.csv'

      mobility_block = (spark
        .read
        .format("CSV")
        .option("header",False)
        .option("delimiter","|")
        .load(mobility_block_path)
        .toPandas())
      mobility_block_totals = list(mobility_block.iloc[0])
      mobility_block.columns = list(mobility_block.iloc[1])
      mobility_block = mobility_block[2:]

      mobility_block_activity = (spark
        .read
        .format("CSV")
        .option("header",False)
        .option("delimiter","|")
        .load(mobility_block_activity_path)
        .toPandas())
      mobility_block_activity_totals = list(mobility_block_activity.iloc[0])
      mobility_block_activity.columns = list(mobility_block_activity.iloc[1])
      mobility_block_activity = mobility_block_activity[2:]

      mobility_unblock = (spark
        .read
        .format("CSV")
        .option("header",True)
        .option("delimiter","|")
        .load(mobility_unblock_path)
        .toPandas())

      whitelist = (spark
        .read
        .format("CSV")
        .option("header",True)
        .option("delimiter","|")
        .load(whitelist_path)
        .toPandas())
      whitelist_total_lines = whitelist.iloc[0][2]
      whitelist.columns = ["Number","cnam","date","_c3"]
      whitelist = whitelist[1:]

      mobility_unblock = (spark
        .read
        .format("CSV")
        .option("header",True)
        .option("delimiter","|")
        .load(mobility_unblock_path)
        .toPandas())

      mobility_flagged1 = (spark
        .read
        .format("CSV")
        .option("header",False)
        .option("delimiter","|")
        .load(mobility_flagged_path1)
        .toPandas())
      mobility_flagged1_totals = list(mobility_flagged1.iloc[0])
      mobility_flagged1.columns = list(mobility_flagged1.iloc[1])
      mobility_flagged1 = mobility_flagged1[2:]
      mobility_flagged1.columns =['Number', 'num_of_OPCs', 'LNP_Lookup', 'num_of_Calls','DNC']

      mobility_flagged2 = (spark
        .read
        .format("CSV")
        .option("header",False)
        .option("delimiter","|")
        .load(mobility_flagged_path2)
        .select("_c0","_c1","_c3","_c4")
        .toPandas())
      mobility_flagged2_totals = list(mobility_flagged2.iloc[0])
      mobility_flagged2.columns = list(mobility_flagged2.iloc[1])
      mobility_flagged2 = mobility_flagged2[2:]
      mobility_flagged2.columns =['Number', 'num_of_OPCs','num_of_Calls','DNC']

      vm_pilot = (spark
        .read
        .format("CSV")
        .option("header",True)
        .option("delimiter",",")
        .load(vm_pilot_path)
        .select("DEPOSIT_NUMBER")  
        .withColumnRenamed("DEPOSIT_NUMBER","ctn")
        .toPandas())

      #combine all numbers to be filtered out, blocked, flagged, or whitelist
      filter_out = ["1"+ctn for ctn in list(set(whitelist.Number)
                                            .union(set(mobility_block.Number))
                                            .union(set(vm_pilot.ctn))
                                            .union(set(mobility_flagged1.Number))
                                            .union(set(mobility_flagged2.Number)))]

      try:
        df = (spark.read.format("csv").option("header",True).load(f"abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/call_graph/source/daily_refresh/{date}/features_extracted/")
        .withColumn("outgoing_call_count", col("outgoing_call_count").cast(IntegerType()))
            .withColumn("incoming_call_count", col("incoming_call_count").cast(IntegerType()))
            .withColumn("incoming_to_outgoing_ratio", col("incoming_to_outgoing_ratio").cast(DoubleType()))
            .withColumn("outgoing_untrusted_call_count", col("outgoing_untrusted_call_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_call_count", col("outgoing_trusted_call_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_call_ratio", col("outgoing_trusted_call_ratio").cast(DoubleType()))
            .withColumn("call_volume", col("call_volume").cast(IntegerType()))
            .withColumn("otc_in_ratio", col("otc_in_ratio").cast(DoubleType()))
            .withColumn("unique_ctn_called", col("unique_ctn_called").cast(IntegerType()))
            .withColumn("mean_degree_one_outgoing_calls", col("mean_degree_one_outgoing_calls").cast(DoubleType()))
            .withColumn("trusted_edge_count", col("trusted_edge_count").cast(IntegerType()))
            .withColumn("outgoing_text_count", col("outgoing_text_count").cast(IntegerType()))
            .withColumn("incoming_text_count", col("incoming_text_count").cast(IntegerType()))
            .withColumn("incoming_to_outgoing_ratio_text", col("incoming_to_outgoing_ratio_text").cast(DoubleType()))
            .withColumn("outgoing_untrusted_text_count", col("outgoing_untrusted_text_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_text_count", col("outgoing_trusted_text_count").cast(IntegerType()))
            .withColumn("outgoing_trusted_text_ratio", col("outgoing_trusted_text_ratio").cast(DoubleType()))
            .withColumn("unique_ctn_texted", col("unique_ctn_texted").cast(IntegerType()))
            .withColumn("mean_degree_one_outgoing_texts", col("mean_degree_one_outgoing_texts").cast(DoubleType()))
            .withColumn("volume_cat", col("volume_cat").cast(IntegerType()))
            .withColumn("repeated_calls", col("repeated_calls").cast(DoubleType()))
            .withColumn("repeated_texts", col("repeated_texts").cast(DoubleType()))
            .withColumn("text_volume", col("text_volume").cast(IntegerType()))
            .withColumn("ott_in_ratio", col("ott_in_ratio").cast(DoubleType()))
            .withColumn("risk_cat", col("risk_cat").cast(IntegerType()))
            .withColumn("Date", date_format(to_date(col("Date")),"yyyyMMdd"))
            ).fillna(0).fillna(0.0).fillna(False)

        df2 = (df
                  .withColumn("whitelist", col("ctn").isin(list(set(whitelist.Number))))
                  .withColumn("blocklist", col("ctn").isin(list(set(mobility_block.Number))))
                  .withColumn("vm_pilot", (col("ctn").isin(list(set(vm_pilot.ctn))) | (col("ctn").substr(1,3) == "999")))
                  .withColumn("flagged", col("ctn").isin(list(set(mobility_flagged1.Number).union(set(mobility_flagged2.Number)))))
                 )

        df2.write.option("header",True).partitionBy("Date").option("mergeSchema", "true").format("delta").mode("append").save(historical_robo_labels_path)
      except Exception as e:
        print(e)


In [0]:
import pandas as pd
from functools import reduce
from pyspark.sql import DataFrame
from datetime import datetime

In [0]:
date1 = '20220128'
date2 = str(datetime.now()).replace("-","")
mydates = pd.date_range(date1, date2).tolist()

# COMMAND ----------

run_date = str(mydates[0].date())
root = "abfss://robofeed@famlisandbox.dfs.core.windows.net/robo_rpts/"
mobility_block_path = root +f"research.{run_date}.MobilityBLOCK"
whitelist_path = root +f"research.{run_date}.Whitelist"
mobility_flagged_path1 = root +f"research.{run_date}.RobocallMobility"
mobility_flagged_path2 = root +f"research.{run_date}.RobocallMobility-8YY"
mobility_unblock_path = root +f"research.{run_date}.MobilityUNBLOCK"
mobility_block_activity_path = root +f"research.{run_date}.MobilityBlockActivity"
vm_pilot_path = 'abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/eh163e/data/robocalling/VM_ID_Pilot_Numbersd.csv'

In [0]:
run_date

Out[6]: '2022-01-29'

In [0]:
whitelist = (spark
  .read
  .format("CSV")
  .option("header",True)
  .option("delimiter","|")
  .load(whitelist_path)
  .toPandas())
whitelist_total_lines = whitelist.iloc[0][2]
whitelist.columns = ["Number","cnam","date","_c3"]
whitelist = whitelist[1:]

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-774409948270981> in <module> 
 ----> 1 whitelist = (spark
 2 . read
 3 . format ( "CSV" ) 
 4 . option ( "header" , True ) 
 5 . option ( "delimiter" , "|" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 156 self . options ( ** options ) 
 157 if isinstance ( path , str ) : 
 --> 158 return self . _df ( self . _jreader . load ( path ) ) 
 159 elif path is not None : 
 160 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o420.load.
: Operation failed: "This request is not authorized to perform this operation using this permission.", 403, HEAD, https://famlisandbox.dfs.core.windows.net/robofeed/robo_rpts/research.2022-01-28.Whitelist?upn=false&action=getStatus&timeout=90
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsRestOperation.execute(AbfsRestOperation.java:246)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsClient.getPathStatus(AbfsClient.java:712)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.getFileStatus(AzureBlobFileSystemStore.java:900)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.getFileStatus(AzureBlobFileSystem.java:682)
	at org.apache.hadoop.fs.FileSystem.isDirectory(FileSystem.java:1777)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:59)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:405)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:356)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:323)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:323)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:236)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

historical_robo_labels_path = "abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/eh163e/data/robocalling/historical_robo_features_with_labels3"

files = dbutils.fs.ls("abfss://famli-dev-workspace@famlisandbox.dfs.core.windows.net/call_graph/source/daily_refresh/")
